<a href="https://colab.research.google.com/github/it-ces/PUBLIC-AI/blob/main/Probability/Simlab/Lab_Hypothesis_bankruptcy(ai_page).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Iván Andrés Trujillo Abella
# ivantrujillo1229@gmail.com

# Hypothesis testing

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats.distributions as dist

In [ ]:
URL = "https://raw.githubusercontent.com/it-ces/Datasets/main/basesuper.csv"

In [ ]:
df = pd.read_csv(URL)

In [ ]:
df

,Unnamed: 0,NIT,y,Ingresos de actividades ordinarias,Costo de ventas,Ganancia bruta,Otros ingresos,Gastos de ventas,Gastos de administración,Otros gastos,...,Prima de emisión,Acciones propias en cartera,Inversión suplementaria al capital asignado,Otras participaciones en el patrimonio,Superavit por revaluación,Otras reservas,Ganancias acumuladas,Patrimonio total,Total de patrimonio y pasivos,_merge
0,0,800009830.0,0,1322093.0,39724.0,1282369.0,14121.0,14483.0,180234.0,81522.0,...,6607.0,NaN,NaN,NaN,NaN,46902.0,5380400.0,5527713.0,6904653.0,both
1,1,800009830.0,0,1322093.0,39724.0,1282369.0,14121.0,14483.0,180234.0,81522.0,...,6607.0,NaN,NaN,NaN,NaN,46902.0,5380400.0,5527713.0,6904653.0,both
2,2,800009830.0,0,1322093.0,39724.0,1282369.0,14121.0,14483.0,180234.0,81522.0,...,6607.0,NaN,NaN,NaN,NaN,46902.0,5380400.0,5527713.0,6904653.0,both
3,3,800009830.0,0,1322093.0,39724.0,1282369.0,14121.0,14483.0,180234.0,81522.0,...,6607.0,NaN,NaN,NaN,NaN,46902.0,5380400.0,5527713.0,6904653.0,both
4,4,800095068.0,0,9514013.0,7554030.0,1959983.0,1665.0,436294.0,537606.0,383833.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3919158.0,4291158.0,8115039.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12768,12768,901202882.0,0,0.0,0.0,0.0,0.0,100.0,170.0,83.0,...,0.0,0.0,NaN,0.0,0.0,0.0,-353.0,647.0,917.0,both
12769,12769,901209458.0,0,0.0,NaN,0.0,0.0,NaN,62450.0,374.0,...,NaN,NaN,NaN,NaN,NaN,NaN,-63454.0,-62454.0,1361958.0,both
12770,12770,901216979.0,0,258644.0,219700.0,38944.0,0.0,NaN,17653.0,1053.0,...,NaN,NaN,NaN,NaN,NaN,NaN,11791.0,36791.0,219013.0,both
12771,12771,901218142.0,0,150529.0,NaN,150529.0,0.0,NaN,52028.0,NaN,...,1509391.0,NaN,NaN,3436642.0,NaN,7042654.0,-15662.0,13871762.0,14788446.0,both


# One sample test for one proportion

According to a news paper the rate of bankruptcy in the country is $30\%$.
thus we stated:
\begin{equation}
H_{0}: P  = 0.3 \\
H_{a}: P \neq 0.3
\end{equation}


In [ ]:
df.y.value_counts(normalize=True)

0    0.986299
1    0.013701
Name: y, dtype: float64

In [ ]:
success = (df.y==1).sum()
P = (df.y==1).mean()
print(P)

0.013700775072418382


In [ ]:
#Getting P-val
P_null = 0.3
se = np.sqrt(P_null* (1-P_null) / len(df))
test_stat  = (P - P_null )/se
p_val = 2 * dist.norm.cdf(-np.abs(test_stat))
print(test_stat, p_val) # P<alpha therefore reject Ho  (Normal approximation)

-70.60849927383533 0.0


In [ ]:
print(sm.stats.proportions_ztest(success, len(df), P_null)) # Normal approximation with null proportion in SE

(-278.3490312870485, 0.0)


In [ ]:
sm.stats.binom_test(success, len(df), P_null)

0.0

# Hypothesis test for two proportions ( Two-Sample test)

We are going to see if there are difference in the proportion of bankrupt among type of society (SAS) agains the other types.

\begin{equation}
H_{0}: P_{sas} = P_{another} \\
H_{1}: P_{sas} \neq P_{another}
\end{equation}

In [ ]:
df['Tipo societario'].unique()

array(['01. SOCIEDAD ANÓNIMA',
       '08. SOCIEDAD POR ACCIONES SIMPLIFICADA SAS',
       '03. SOCIEDAD LIMITADA', '07. EMPRESA UNIPERSONAL',
       '04. SOCIEDAD EN COMANDITA',
       '05. SOCIEDAD EN COMANDITA POR ACCIONES',
       '02. SUCURSAL EXTRANJERA', '06. COLECTIVA'], dtype=object)

In [ ]:
df  = df[['y', 'Tipo societario']].dropna()
df['Tipo societario']  = np.where(df['Tipo societario']=='08. SOCIEDAD POR ACCIONES SIMPLIFICADA SAS','sas','other')

In [ ]:
p = df.groupby("Tipo societario")["y"].agg([lambda x: np.mean(x==1), "size"])
p.columns = ['proportion', 'n']
p

,proportion,n
Tipo societario,,
other,0.015032,5056
sas,0.012829,7717


In [ ]:
## The pooled approach
p_common  = (df.y==1).mean()
va = p_common * (1 - p_common)
se = np.sqrt(va*( 1 / p.n.other + 1/p.n.sas))
test_stat = (p.proportion.sas - p.proportion.other)/se
pval= 2 * dist.norm.cdf(-np.abs(test_stat))
print(test_stat, pval) # We cannot reject the null-hypothesis.

-1.0473326005760872 0.29494620527493454


In [ ]:
# With a single command!
df_sas = df.loc[df['Tipo societario'] == "sas", "y"]
df_another = df.loc[df['Tipo societario'] == "other", "y"]
print(sm.stats.ttest_ind(df_sas, df_another))

(-1.0472955717580241, 0.294983097897839, 12771.0)


# Comparing with mean

According to a research the mean of "activos corrientes totales" in the colombian firms is 12.5 millons

\begin{equation}
H_{0}: \mu = 12.5 \\
H_{a} : \mu \neq 12.5
\end{equation}


In [ ]:
df = pd.read_csv(URL)
df[ 'Activos corrientes totales'] = df[ 'Activos corrientes totales']/1000000

In [ ]:
sm.stats.ztest(df['Activos corrientes totales'], value=12.5)  # prints test statistic, p-value Reject Ho.

(-17.866189423661616, 2.1627255646136666e-71)

# Comparing two-means

Research question:
There is signficativive difference among bankrupty and non-bankrupt firms in "Activos corrientes totales"?

\begin{equation}
H_{0} : \mu_{bankrup} - \mu_{noBankrupt}  = 0 \\
H_{1} : \mu_{bankrup} - \mu_{noBankrupt}  \neq 0
\end{equation}

A important note here: you could perfomr the test by z-test ot t-test both approaches are similar when n tend to be large.


In [ ]:
df = df[['Activos corrientes totales', 'y']].dropna()

In [ ]:
df_bankrupt =  df.loc[df.y==1, 'Activos corrientes totales']
df_nobankrupt =  df.loc[df.y==0, 'Activos corrientes totales']

In [ ]:
print(df_bankrupt.mean(), df_nobankrupt.mean())

11.814262525714286 9.315362009842831


In [ ]:
print(sm.stats.ztest(df_bankrupt, df_nobankrupt)) #  test statistic, p value
print(sm.stats.ttest_ind(df_bankrupt, df_nobankrupt)) # test statistic, p value, degrees of freedom
# What is your alpha? reject or not?

(1.647483618800559, 0.09945868054271348)
(1.6474836188005564, 0.09948328179275195, 12771.0)


# Paired test

Is when we have the same unit of measure prior and posterior an intervention, this is very important to assess influential or procedures over patients or reform in firms ( two repeated measurements on the same unit).


In [ ]:
# We have collected a medical baseline variable for a set of patiens after and before of the consumption of  drug
data = {'ind':     [1,2,3,4,5,6,7,8,9,10],
        'vart1':   [12, 33, 13,  44,  33,   45,12,  33.3, 43.2, 8.8],
        'vart2':   [13, 33.9,  12,  44.5, 44.1, 45.9, 12.4,35.6,  44, 9.4  ]}
data = pd.DataFrame(data)

In [ ]:
data # We want to know if there are significative difference after take the drug..

,ind,vart1,vart2
0,1,12.0,13.0
1,2,33.0,33.9
2,3,13.0,12.0
3,4,44.0,44.5
4,5,33.0,44.1
5,6,45.0,45.9
6,7,12.0,12.4
7,8,33.3,35.6
8,9,43.2,44.0
9,10,8.8,9.4


In [ ]:
data['diff'] = data['vart2'] - data['vart1']
print(data['diff'].describe())
print(sm.stats.ztest(data['diff']))
# is as used a one sample test to compare the difference to zero..

count    10.000000
mean      1.750000
std       3.381403
min      -1.000000
25%       0.525000
50%       0.850000
75%       0.975000
max      11.100000
Name: diff, dtype: float64
(1.636594415483862, 0.1017152432692071)
